In [1]:
!pip install -q --upgrade xlrd openpyxl


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

df = pd.read_excel(r'C:\Users\Acer\github\Credit_Card_Clients\Data\Credit Card Clients-20251001T113651Z-1-001\Credit Card Clients\default of credit card clients.xls', 
                   header=1)

print("Dados para dataset original do Cartão de crédito:")
display(df.head(10))

KeyboardInterrupt: 

In [ ]:
print(df.columns.tolist())

In [ ]:
aux_df = df.copy()

df_reduzido = (df
               .assign(
                   BILL_MEAN = df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].mean(axis=1),
                   PAY_MEAN  = df[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']].mean(axis=1)
               )
               .drop(columns=['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6',
                              'PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6', 'default payment next month'])
              )

df_reduzido['DEFAULT_PAYMENT_NEXT_MONTH'] = aux_df['default payment next month']

print("Dataset como as medias dos pagamentos e faturas:")
display(df_reduzido.head(10))          

In [ ]:
print(df_reduzido.isnull().sum())

In [ ]:
print(df_reduzido.duplicated().sum())

In [ ]:
print(df_reduzido['EDUCATION'].value_counts())
print()
print(df_reduzido['MARRIAGE'].value_counts())
print()
print(df_reduzido['SEX'].value_counts())
print()
print(df_reduzido['AGE'].value_counts())

In [ ]:
df_reduzido['EDUCATION'] = df_reduzido['EDUCATION'].replace([0,5,6], 4)
df_reduzido['MARRIAGE'] = df_reduzido['MARRIAGE'].replace([0], 3)

print(df_reduzido['EDUCATION'].value_counts())
print()
print(df_reduzido['MARRIAGE'].value_counts())

display(df_reduzido.head(10))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_normalized = df_reduzido.copy()

df_normalized[['BILL_MEAN','PAY_MEAN']] = scaler.fit_transform(df_normalized[['BILL_MEAN','PAY_MEAN']])

display(df_normalized.head(10))

In [ ]:
bins = [18, 29, 49, 59, 69, 130]
labels = [1, 2, 3, 4, 5]

# Criar AGE_GROUP no df_normalized
df_normalized['AGE_GROUP'] = pd.cut(df_normalized['AGE'], bins=bins, labels=labels, right=True)
df_normalized['AGE_GROUP'] = df_normalized['AGE_GROUP'].astype(int)

# Conferir
print(df_normalized['AGE_GROUP'].value_counts())
display(df_normalized)

In [ ]:
bins = [18, 28, 48, 58, 68, 130]  
labels = [1, 2, 3, 4, 5]   

df_normalized['AGE_GROUP'] = pd.cut(df_normalized['AGE'], bins=bins, labels=labels, right=True)
df_normalized['AGE_GROUP'] = df_normalized['AGE_GROUP'].astype(int)

display(df_normalized[['AGE','AGE_GROUP']].head(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Segmenta os dados para DEFAULT = 0
df_normalized[df_normalized['DEFAULT_PAYMENT_NEXT_MONTH'] == 0]['AGE'].hist(
    alpha=0.5, color='blue', bins=len(df_normalized['AGE'].unique())
)

# Segmenta os dados para DEFAULT = 1
df_normalized[df_normalized['DEFAULT_PAYMENT_NEXT_MONTH'] == 1]['AGE'].hist(
    alpha=0.7, color='red', bins=len(df_normalized['AGE'].unique())
)

# Adiciona legenda e títulos
plt.xlabel('Faixa Etária')
plt.ylabel('Quantidade')
plt.title('Histograma de idade por quantidade')
plt.legend(['Conta paga', 'Conta não paga'])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.stripplot(data=df_normalized, x='AGE_GROUP', y='BILL_MEAN', hue='DEFAULT_PAYMENT_NEXT_MONTH', jitter=True, palette='Set1', alpha=0.7)
plt.title('Dispersão de BILL_MEAN por Faixa Etária e Default')
plt.xlabel('Faixa Etária')
plt.ylabel('Pagamento de fatura (normalizado)')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
sns.boxplot(data=df_normalized, x='AGE_GROUP', y='BILL_MEAN', hue='DEFAULT_PAYMENT_NEXT_MONTH', palette='Set2')
plt.title('Boxplot da fatura por faixa etária e Default')
plt.xlabel('Faixa Etária (AGE_GROUP)')
plt.ylabel('BILL_MEAN (normalizado)')
plt.show()